[Source](https://colab.research.google.com/drive/1BiQiw31DT7-cDp1-0ySXvvhzqomTdI-o?usp=sharing&pli=1&authuser=5#scrollTo=_kbS7nRxcMt7)

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6

In [3]:
import datasets
from datasets import load_dataset
dataset_name = "OpenAssistant/oasst1"
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

In [4]:
#convert the dataset to pandas dataframe
df = dataset.to_pandas()
assistances = df[(df['role'] == 'assistant') & (df['rank'] == 0.0)]
prompters = df[df['role'] == 'prompter']
prompters.set_index('message_id',inplace=True)
assistant_prompt_values = []
for _,row in assistances.iterrows():
  prompt_text = prompters.loc[row.parent_id,'text']
  new_row = "### Human: " + prompt_text + "### Assistant: " + row.text
  assistant_prompt_values.append(new_row)

assistances['prompt_assistance_text'] = assistant_prompt_values
# convert into HG dataset
finetuning_dataset = datasets.Dataset.from_pandas(assistances)
finetuning_dataset['prompt_assistance_text'][:3]

<ipython-input-4-7ad9384e37c7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assistances['prompt_assistance_text'] = assistant_prompt_values


['### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, lea

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "microsoft/phi-2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
print(model)

PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (out_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
        )
      )

In [8]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,

    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "Wqkv",
        "fc1",
        "fc2"
    ]
)

In [9]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    #gradient_checkpointing=True,
)

In [10]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
    model=model,
    train_dataset=finetuning_dataset,
    peft_config=peft_config,
    dataset_text_field="prompt_assistance_text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/17972 [00:00<?, ? examples/s]

In [11]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [12]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.727000
20,1.729200
30,1.741600
40,2.015800
50,2.143000
60,1.646900
70,1.642300
80,1.666900
90,1.919000
100,2.155700


TrainOutput(global_step=1000, training_loss=1.7569399700164794, metrics={'train_runtime': 2327.0056, 'train_samples_per_second': 6.876, 'train_steps_per_second': 0.43, 'total_flos': 5.191103660531712e+16, 'train_loss': 1.7569399700164794, 'epoch': 0.89})

In [13]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [14]:
# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<s>[INST] What is a large language model? [/INST]>

<s>[INST] What is a large language model?[/INST]>

A large language model is a type of artificial intelligence model that is trained on a massive amount of text data. It is designed to be able to generate human-like text, and can be used for a variety of applications, such as generating text, summarizing text, and even generating images.

Large language models are typically trained on large datasets, such as Wikipedia or other text corpora. They are then trained using a deep neural network architecture, such as a transformer, which allows them to learn the relationships between words and phrases in the text.

Large language models are often used in conjunction with other AI models, such as natural language processing models, to create more advanced AI systems. They are also used in a variety of applications, such as chatbots, virtual assistants, and even in the development of


In [16]:
# Run text generation pipeline with our next model
prompt = "What is a model explainability?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is a model explainability? [/INST]<s>What is a model explainability?</s>

<s>Model explainability is the ability to understand how a model makes decisions and predictions. It is important because it helps us understand how the model works and how to improve it. It also helps us understand why the model made a certain decision or prediction, which can be useful in many applications.</s>

<s>What are some examples of model explainability?</s>

<s>Some examples of model explainability include:

- Understanding how a neural network makes predictions
- Understanding how a decision tree makes decisions
- Understanding how a linear regression model makes predictions
- Understanding how a random forest makes predictions
- Understanding how a support vector machine makes predictions
- Understanding how a deep learning model makes predictions
- Understanding how a reinforcement learning model makes decisions
- Understanding how a reinforcement learning model makes decisions


In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [18]:
ls


gdrive/  results/  sample_data/  wandb/


In [20]:
cd results


/content/results


In [24]:
ls

checkpoint-10/    checkpoint-280/  checkpoint-470/  checkpoint-660/  checkpoint-850/
checkpoint-100/   checkpoint-290/  checkpoint-480/  checkpoint-670/  checkpoint-860/
checkpoint-1000/  checkpoint-30/   checkpoint-490/  checkpoint-680/  checkpoint-870/
checkpoint-110/   checkpoint-300/  checkpoint-50/   checkpoint-690/  checkpoint-880/
checkpoint-120/   checkpoint-310/  checkpoint-500/  checkpoint-70/   checkpoint-890/
checkpoint-130/   checkpoint-320/  checkpoint-510/  checkpoint-700/  checkpoint-90/
checkpoint-140/   checkpoint-330/  checkpoint-520/  checkpoint-710/  checkpoint-900/
checkpoint-150/   checkpoint-340/  checkpoint-530/  checkpoint-720/  checkpoint-910/
checkpoint-160/   checkpoint-350/  checkpoint-540/  checkpoint-730/  checkpoint-920/
checkpoint-170/   checkpoint-360/  checkpoint-550/  checkpoint-740/  checkpoint-930/
checkpoint-180/   checkpoint-370/  checkpoint-560/  checkpoint-750/  checkpoint-940/
checkpoint-190/   checkpoint-380/  checkpoint-570/  checkpoint-760

In [27]:
cp -r checkpoint-960/ ../gdrive/MyDrive/

In [21]:
ls

checkpoint-10/    checkpoint-280/  checkpoint-470/  checkpoint-660/  checkpoint-850/
checkpoint-100/   checkpoint-290/  checkpoint-480/  checkpoint-670/  checkpoint-860/
checkpoint-1000/  checkpoint-30/   checkpoint-490/  checkpoint-680/  checkpoint-870/
checkpoint-110/   checkpoint-300/  checkpoint-50/   checkpoint-690/  checkpoint-880/
checkpoint-120/   checkpoint-310/  checkpoint-500/  checkpoint-70/   checkpoint-890/
checkpoint-130/   checkpoint-320/  checkpoint-510/  checkpoint-700/  checkpoint-90/
checkpoint-140/   checkpoint-330/  checkpoint-520/  checkpoint-710/  checkpoint-900/
checkpoint-150/   checkpoint-340/  checkpoint-530/  checkpoint-720/  checkpoint-910/
checkpoint-160/   checkpoint-350/  checkpoint-540/  checkpoint-730/  checkpoint-920/
checkpoint-170/   checkpoint-360/  checkpoint-550/  checkpoint-740/  checkpoint-930/
checkpoint-180/   checkpoint-370/  checkpoint-560/  checkpoint-750/  checkpoint-940/
checkpoint-190/   checkpoint-380/  checkpoint-570/  checkpoint-760

In [26]:
ls ../gdrive/MyDrive/

'Colab Notebooks'/
